<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/intel/Neural_speed_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
!pip install  mlflow pyngrok evaluate  bitsandbytes accelerate datasets transformers==4.41.2 tiktoken gguf transformers_stream_generator einops --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00


#Pytorch Model from Hugginface

In [15]:
from transformers import AutoTokenizer, TextStreamer, AutoModelForCausalLM

model_name = "Intel/neural-chat-7b-v3-1"     # Hugging Face model_id or local model
prompt = "Once upon a time, there existed a little girl,"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
inputs = tokenizer(prompt, return_tensors="pt").input_ids
streamer = TextStreamer(tokenizer)


model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True)
model.generation_config.pad_token_id = tokenizer.pad_token_id

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
import datetime
t1 = datetime.datetime.now()

outputs = model.generate(inputs, streamer=streamer, max_new_tokens=300, pad_token_id=tokenizer.eos_token_id)

t2 = datetime.datetime.now()


<s> Once upon a time, there existed a little 

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


girl, who was very curious about the world around her. She was always eager to learn new things and explore new places.

One day, she heard about a magical land called “The Land of Enchantment.” It was said to be filled with beautiful castles, enchanted forests, and friendly creatures. The little girl was so excited to visit this magical place.

She decided to embark on a journey to The Land of Enchantment. She packed her bags, filled them with her favorite books, and set out on her adventure.

As she traveled, she encountered many obstacles and challenges. She met a wise old man who taught her how to navigate through the enchanted forests. She also met a kind fairy who shared her knowledge of the magical creatures that lived in the land.

The little girl learned to be brave and resourceful, using her newfound skills to overcome the obstacles she faced. She also made many new friends along the way, who shared their own stories and experiences.

Finally, after many days of traveling, th

In [18]:
(t2-t1).seconds, len(outputs[0])

(27, 312)

In [19]:
len(outputs[0])/ (t2-t1).seconds

11.555555555555555

# GGUF Model from Hugging Face


In [24]:
import gc
import torch
# model.cpu()
# del model
gc.collect()
torch.cuda.empty_cache()

In [25]:
# Specify the GGUF repo on the Hugginface
model_name = "TheBloke/Llama-2-7B-Chat-GGUF"
# Download the the specific gguf model file from the above repo
gguf_file = "llama-2-7b-chat.Q4_0.gguf"
# make sure you are granted to access this model on the Huggingface.
tokenizer_name = "meta-llama/Llama-2-7b-chat-hf"

prompt = "Once upon a time"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)
inputs = tokenizer(prompt, return_tensors="pt").input_ids
streamer = TextStreamer(tokenizer)


model = AutoModelForCausalLM.from_pretrained(model_name, gguf_file = gguf_file, force_download=True)
model.generation_config.pad_token_id = tokenizer.pad_token_id

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


llama-2-7b-chat.Q4_0.gguf:   0%|          | 0.00/3.83G [00:00<?, ?B/s]

llama-2-7b-chat.Q4_0.gguf:   0%|          | 0.00/3.83G [00:00<?, ?B/s]

Converting and de-quantizing GGUF tensors...: 100%|██████████| 291/291 [00:47<00:00,  6.14it/s]


In [26]:
t1 = datetime.datetime.now()

outputs = model.generate(inputs, streamer=streamer, max_new_tokens=300, pad_token_id=tokenizer.eos_token_id)

t2 = datetime.datetime.now()

print((t2-t1).seconds)

<s> Once upon a time, in a land far, far, far away, far, far away, far, far away, far away, far away, there, there, there, thereout, there, away land,ere’orthy’alies,,
Љadies away away away
Љase
Љase,
Љy [ kwiet [ly [[ly [[[[[[[[[[the, [the,the,the [the the the [the
Љs’the.the’s’s’s’s’dpts’d’d’dpts'dpts'dpts'd'd'd'dpts'd hath hath hath hath hathptsptsôppalppalЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉЉ
Љnono everybody();`();` everybody everybody everybody everybody everybody everybody everybody everybody everybody everybody everybody everybody everybody everybodyO nobodyO nobodyO nobodyO\\O\\O\\O\\O\\OOOOOOOOOOOOOOOOOOOOOOOOOOOO<s> TagsO<s> (O<s> (O<s>Љ<s>Љ<s>Љ<s>
194


In [27]:
(t2-t1).seconds, len(outputs[0])

(194, 305)

In [28]:
len(outputs[0])/ (t2-t1).seconds

1.5721649484536082

# PyTorch Model qwen/Qwen-7B



In [33]:

from transformers import AutoTokenizer, TextStreamer, AutoModelForCausalLM

model_name = "qwen/Qwen-7B"     #
prompt = "Once upon a time, there existed a little girl,"

model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True,  trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
inputs = tokenizer(prompt, return_tensors="pt").input_ids
model.generation_config.pad_token_id = tokenizer.pad_token_id

streamer = TextStreamer(tokenizer)

qwen_generation_utils.py:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/qwen/Qwen-7B:
- qwen_generation_utils.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


cpp_kernels.py:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/qwen/Qwen-7B:
- cpp_kernels.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/qwen/Qwen-7B:
- qwen_generation_utils.py
- cpp_kernels.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/19.5k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

tokenization_qwen.py:   0%|          | 0.00/9.62k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/qwen/Qwen-7B:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


qwen.tiktoken:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

In [35]:
import datetime
t1 = datetime.datetime.now()

outputs = model.generate(inputs, streamer=streamer, max_new_tokens=300, pad_token_id=tokenizer.eos_token_id)

t2 = datetime.datetime.now()

print((t2-t1).seconds)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Once upon a time, there existed a little 

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


girl, who shall remain nameless, who was very enthusiastic about the idea of growing things. Her family, however, did not share this enthusiasm. Their house was in the middle of the city, where you could only buy vegetables that had been transported great distances and were expensive and boring. This little girl had to watch as her friends ate the delicious fresh vegetables from their gardens while she had only the taste of the past in her mouth. How could she get her family to eat the vegetables she grew?
A very little girl grew vegetables in a pot. They were delicious! When she served them to her family, they complained that they were too salty and too watery. They did not want to eat the vegetables.
The little girl could not bear it. She went to her grandpa and asked him what to do. He said that she had to use the vegetable mixture as a gravy for their dinner. She did so, and it was delicious!
The next day, the little girl had to give the vegetable mixture to her family for dinner a

In [37]:
(t2-t1).seconds, len(outputs[0])

(26, 311)

In [38]:
len(outputs[0])/ (t2-t1).seconds

11.961538461538462